## Staph data
http://users-birc.au.dk/cstorm/courses/MLiB_f14/project1.html

In [78]:
import requests
import numpy as np
import pandas as pd
import re
from itertools import groupby
import pickle

In [2]:
staph_url = ["http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome1.fa",
            "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome2.fa",
           "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/genome3.fa"]
staph_annot = ["http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation1.fa",
              "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation2.fa"
              "http://users-birc.au.dk/cstorm/courses/MLiB_f14/projects/project3/annotation3.fa"]

## Load/ Modify Data

In [266]:
def load_file(url,genome=True):
    """input: url for genome, 
    output: pulled data without header and as a single txt string"""
    data = ''.join(requests.get(url).text.split("\n")[1::])
    return data

def segment_genome(data_string,split_size):
    """input: genome data string (data_string) and then size we should split the data (split_size)
    output: array split by specified split_size"""
    chunks = len(data_string)
    data = [data_string[i:i+split_size] for i in range(0,chunks,split_size)]
    return data

def gene_label(annotation, min_gene=10):
    """Input: annotation
    ouput: the label array for data (true if C or R else false for > 50% of line)"""
    size = len(annotation[0])
    print(size)
    label = [True if len(line.strip("C")) < size/2 or len(line.strip("R")) < size/2 else False 
             for line in annotation]
    return label

def gene_label_dir(annotation,direction="C"):
    """Input: annotation and direction is C (forward) unless specified as R (reverse)
    ouput: the label array for data"""
    size = len(annotation[0])
    label = [True if len(line.strip(direction)) < size/2 else False 
             for line in annotation]
    return label

def nucleotide_frequency(seq):
    '''Count the occurrences of characters in "seq".'''
    counts = {'A':0,'C':0,'G':0,'T':0}
    for c in seq:
        counts[c] +=1
    total = sum(list(counts.values()))
    for key, value in counts.items():
        counts[key] = value / total
    return counts

In [267]:
import re

def ORF_finds(seq):
    """input: A sequence string (the genome) is taken and all the ATG's in the sequence are found
    output: a list of start indices 
    *** on this reverse strand... these are from the end of the sequence!!"""
    starts = [m.start() for m in re.finditer('ATG', seq)]
    return starts

def sequence_list(genome,ends,start):
    """input: output: """
    sequences = [genome[start:start+end+3] for end in ends[0]]
    sequences += [genome[start:start+end+3] for end in ends[1]] 
    sequences += [genome[start:start+end+3] for end in ends[2]]
    return sequences

def sequence_list2(genome,ends,start,direction):
    """input: """
    frame = start % 3
    sequences = [(start, start+end+3, 'taa', frame, direction) for end in ends[0]]
    sequences += [(start, start+end+3, 'tag', frame, direction) for end in ends[1]] 
    sequences += [(start, start+end+3, 'tga', frame, direction) for end in ends[2]]
    return sequences

# def geneList(genome,direction):
#     """input: genome (character string), which direction we are looking at (the forward or reverse strand)
#     output: a list of sequences on that particular strand of interest (forward or reverse complement)"""
#     if direction == 'R':
#         genome = reverse_complement(genome)
#     starts = ORF_finds(genome)
#     sequences = []
#     for start in starts:
#         ## m.start()+1 so that when we sample the sequence we go from 
#         ends = [[m.start() for m in re.finditer(x, genome[start:start+2000]) if m.start() % 3 == 0] for x in ["TAA", "TAG", "TGA"]]
#         if direction == 'R':
#             length = len(genome)
#             start = length - 3 - start  # CAT ~ ATG (length - 1 - 2)
#             ends = [[length - e for e in end] for end in ends]
#         sequences.append(sequence_list2(genome,ends,start,direction))
#     return sequences 

def geneList(genome,direction):
    """input: genome (character string), which direction we are looking at (the forward or reverse strand)
    output: a list of sequences on that particular strand of interest (forward or reverse complement)"""
    starts = ORF_finds(genome)
    sequences = []
    for start in starts:
        ## m.start()+1 so that when we sample the sequence we go from 
        ends = [[m.start() for m in re.finditer(x, genome[start:start+2000]) if m.start() % 3 == 0] for x in ["TAA", "TAG", "TGA"]]
        sequences.append(sequence_list2(genome,ends,start,direction))
    return sequences   

def fixed(possible_reverse_sequences, genome_length):
    fixed_reversed_sequences = []
    for possible_reverse in possible_reverse_sequences:
        f = [(genome_length - end , genome_length - start,end_codon,frame,strand) for start,end,end_codon,frame,strand in possible_reverse]
        fixed_reversed_sequences.append(f)
    return fixed_reversed_sequences

def potentialGenes(genome):
    """input: genome which is a string of a's, t's, g's, c's
    ouput: the number of possible genes that start with ATG and 
    end with taa,tag, or tga on the forward and reverse complement DNA strand"""
    forwards = geneList(genome,'F')
    reversed_genome = reverse_complement(str(genome))
    reverse = geneList(reversed_genome,'R')
    reverses = fixed(reverse, len(genome))
    combos = forwards + reverses
    return combos

In [268]:
def reverse_complement(genome):
    """input: genome, output: reverse complement"""
    switch = {"A":"T","G":"C","T":"A","C":"G"}
    rc = ''.join([switch[letter] for letter in genome])[::-1]
    return rc

In [269]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA","TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"X":["TAA", "TAG", "TGA"]}

def invert(d):
    return dict( (v,k) for k in d for v in d[k] )

codon_to_aa = invert(codon_map)

def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq

In [270]:
## amino acid to nucleotide chart http://www.cbs.dtu.dk/courses/27619/codon.html
## X === stop
aa = ["I","L","V","F","M","C","A","G","P","T","S","Y","W","Q","N","H","E","D","K","R","X"]
aa_name = ["Isoleucine","Leucine","Valine","Phenylalanine","Methionine","Cysteine","Alanine",
           "Glycine","Proline","Threonine","Serine","Tyrosine","Tryptophan",
           "Glutamine","Asparagine","Histidine","Glutamic_acid", "Aspartic_acid", 
           "Lysine","Arginine","Stop_codons"]
codons = [["ATT", "ATC", "ATA"],["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          ["GTT", "GTC", "GTA", "GTG"],["TTT", "TTC"],["ATG"],["TGT", "TGC"],
          ["GCT", "GCC", "GCA", "GCG"],["GGT", "GGC", "GGA", "GGG"],
          ["CCT", "CCC", "CCA", "CCG"],
          ["ACT", "ACC", "ACA", "ACG"],["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          ["TAT", "TAC"],["TGG"],["CAA", "CAG"],["AAT", "AAC"],["CAT", "CAC"],["GAA", "GAG"],
          ["GAT", "GAC"],["AAA", "AAG"],["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
          ["TAA", "TAG", "TGA"]]

codon_map = {"I":["ATT", "ATC", "ATA"],
             "L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
             "V":["GTT", "GTC", "GTA", "GTG"],
             "F":["TTT", "TTC"],"M":["ATG"],
             "C":["TGT", "TGC"],
             "A":["GCT", "GCC", "GCA", "GCG"],
             "G":["GGT", "GGC", "GGA", "GGG"],
             "P":["CCT", "CCC", "CCA", "CCG"],
             "T":["ACT", "ACC", "ACA", "ACG"],
             "S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
             "Y":["TAT", "TAC"],
             "W":["TGG"],
             "Q":["CAA", "CAG"],
             "N":["AAT", "AAC"],
             "H":["CAT", "CAC"],
             "E":["GAA", "GAG"],
             "D":["GAT", "GAC"],
             "K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
             "Stop":["TAA", "TAG", "TGA"]}

codon_properties_map = {}

In [271]:
## Thermofisher aa properties table
from lxml import html
def pull_page(url):
    data = requests.get(url).text
    return data
tag = "table table-bordered table-striped"
tag = "table"
url = "https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html"

page = html.fromstring(pull_page(url))
tables = page.cssselect(tag)
aa_table = pd.read_html(html.tostring(tables[0]),header=0)[0]


### aa_properties
Aliphatic: Alanine,Isoleucine,Leucine,Valine
Aromatic: Phenylalanine, Tryptophan, Tyrosine
Polar_neutral: Asparagine, Cysteine, Glutamine, Methionine,Serine,Threonine
charged: Aspartic acid, Glutamic acid
    
Properties table
https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html

http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3456822/pdf/10867_2004_Article_396406.pdf

http://www.proteinstructures.com/Structure/Structure/amino-acids.html

In [272]:
# flipped_annot = annot[::-1]

def check_gene_state(loc,annot,strand):
    """input: output: """
    start, end = int(loc[0]), int(loc[1])
    fwd = list(set(annot[start:end+1]))
    bhd = list(set(annot[start - 1:end]))
    if strand == 'F':
        states = list(set(annot[start:end]))
        state = 'C'
    else:
        r_annot = flipped_annot
        states = list(set(r_annot[start:end]))
        state = 'R'
    if len(states) == 1 and states[0] == state and len(fwd) != 1 and len(bhd) != 1:
        return True
    else:
        return False

In [273]:
def build_training_data(annot, data):
    """This finds all the genes, non-coding regions, 
    reverse regions in the genome and returns them as a list
    annot is the annotated genome with c's as forward genes, 
    n's as noncoding regions and r's as genes on the reverse complement strand. 
    data is the genome comprised of a's,t's,g's,c's. 
    Both annot and data are strings"""
    d = []
    i = 0
    c = ''
    start = 0
    while i < len(annot)-1:
        if str(annot[i]) != str(annot[i+1]):
            end = i
            d.append((start,end,str(c),str(annot[i-1])))
            c = ''
            start = i+1
        else:
            c = c + data[i]
        i += 1
    d.append((start,i,str(c),str(annot[i-1])))   
    return d

In [274]:
from itertools import groupby

def find_regions(annot,data):
    """This finds all the genes, non-coding regions, 
    reverse regions in the genome and returns them as a list
    annot is the annotated genome with c's as forward genes, 
    n's as noncoding regions and r's as genes on the reverse complement strand. 
    data is the genome comprised of a's,t's,g's,c's. 
    Both annot and data are strings"""
    stateGroups = groupby(enumerate(annot), lambda value: value[1])
    genome_regions = [(state, [x for x, _ in iterator]) for state, iterator in stateGroups] 
    regions = [(region[0],min(region[1]),max(region[1])+1,
                data[min(region[1]):max(region[1])+1]) for region in genome_regions]
    return regions

In [275]:
def frameShift(seq,shift):
    """input: seq and shift -- shift not used...."""
    new_seq = ''
    for j, i in enumerate(seq):
        if j % shift == 0:
            new_seq = new_seq + i + ' '
        else: 
            new_seq = new_seq + i
    return new_seq


# fs = [frameShift(seq,3) for seq in gene_df.sequence]

In [276]:
def state(annot):
    """input: states are possible annotations C forward gene,
    N - not a gene, or R for reverse gene"""
    states = list(set(annot))
    return states

def transitions(annot,state):
    """input: determine when transitions occur where annot is annotated 
    and state is the state i'm looking to transition from ouput: there location"""
    trs = list(set(annot))
    transits = {}
    total = len(annot) 

    for transition_state in trs:
        count = len([ts for i,ts in enumerate(annot[:-1]) if str(ts) == state and str(annot[i+1]) == transition_state])
        transits[str(transition_state)] = count/total
    return transits

def emissions(genome,annot,state):
    """input: genome is  a genome, annot is its annotation, 
    state is the state i want to count nucleotide frequency of"""
    ems = list(set(genome))
    
    emissions = {}
    transition_totals = {}

    total = len([letter for letter in annot if str(letter) == state])
    for emission_state in ems:
        count = len([letter for i,letter in enumerate(genome) if 
                     str(letter) == emission_state and str(annot[i])==state])
        emissions[str(emission_state)] = count/total    
    return emissions

def prior_state_emissions(genome,annot):
    """input: """
    state_dictionary = {}
    states = state(annot)
    for s in states:
        if s == 'N':
            prob = 1
        else:
            prob = 0
        em = emissions(genome,annot,s)
        tr = transitions(annot,s)
        state_dictionary[s] = [str(s),prob,em,tr]
    return state_dictionary

In [277]:
# from collections import defaultdict
def aa_find(seq):
    """find aa's in sequence
    input: sequence, 
    output: dictionary of codons in sequence"""
    codon_count = {}
    for aa, codons in codon_map.items():
        codon_count[aa] = 0
        if any(x in seq for x in codons):
            codon_count[aa] += 1
    return codon_count

In [278]:
### Pipeline

## load genome annotation
annot = load_file(staph_annot[0])

## load genome
data = load_file(staph_url[0])

## Built table of sequences for each state in genome
region = find_regions(annot,data)
# re = build_training_data(annot,data)
regions = pd.DataFrame(region)
regions.columns = [["state","start","end","seq"]]
regions.head()
print("Built table of sequences for each state in genome...")
print(regions.head(5))

## Reverse Complement strands found on the Reverse strand
regions.seq = regions.apply(lambda x: reverse_complement(x[3]) if x[0] == "R" else x[3],axis=1)
print("State 'R' sequences were reverse complemented...")

Built table of sequences for each state in genome...
  state  start   end                                                seq
0     N      0   231  TTGTTGATATTCTGTTTTTTCTTTTTTAGTTTTCCACATGAAAAAT...
1     C    231  1587  ATGACTGAAAATGAACAAATTTTTTGGAACAGGGTCTTGGAATTAG...
2     N   1587  1741  CATGTGGAAAAGAATATCTTTTATGAAATAGTTATCCACAAGTTGT...
3     C   1741  2878  ATGATTCAATTTTCAATTAATCGCACATTATTTATTCATGCTTTAA...
4     N   2878  2952  GTAAGAAAAAGCTCCCTTTTAGGAGTTTTTTTGTTATTATAAATAT...
State 'R' sequences were reverse complemented...


In [279]:
## Save Data, corresponding Annotation
def pickle_save(filename, variable):
    filename = filename+'.pkl'
    output = open(filename, 'wb')
    pickle.dump(variable, output)
    output.close()
    print(filename + " Saved!")
    return True

pickle_save('StaphGenome1', data)
pickle_save('StaphGenome1_annot', annot)
pickle_save('StaphGenome1_regions_df', regions)

StaphGenome1.pkl Saved!
StaphGenome1_annot.pkl Saved!
StaphGenome1_regions_df.pkl Saved!


True

In [280]:
import pandas as pd
import os

## Open Pickles in Folder
def find_pickles():
    directory_files = os.listdir()
    filenames = []
    for file in directory_files:
        filename, file_extension = os.path.splitext(file)
        if file_extension in ['.pkl','.p']:
            filenames.append(file)
    return filenames

def open_pickles():
    filenames = find_pickles()
    d = {}
    for file in filenames:
        key = os.path.splitext(file)[0]
        d[key] = pickle_open(file)
    return d

def pickle_open(filename, df=False):
    """Eventually check if we are opening a python 3 or python 2 file 
    and deal with discrepencies (python 3 pickle opened in python 2 etc...)"""
    if df == True:
        file = pd.read_pickle(filename)
        print(filename + " dataframe sucessfully loaded!")
    else:
        file = pickle.load( open( filename, "rb" ) )
        print(filename + " sucessfully loaded!")
    return file

df_test = pickle_open('StaphGenome1_regions_df.pkl',df=True)
data2 = pickle_open('StaphGenome1.pkl')

StaphGenome1_regions_df.pkl dataframe sucessfully loaded!
StaphGenome1.pkl sucessfully loaded!


In [281]:
regions.state.value_counts()

N    1449
C     809
R     639
Name: state, dtype: int64

In [19]:
## Find potential sequences
pot_sequences = potentialGenes(data)
print("Found potential sequences...")

## Create DataFrame
df = pd.DataFrame(list(pd.DataFrame(pot_sequences).unstack().dropna()))
df.columns=[["start","end","end_codon","shift","strand"]]
print("Genome DataFrame Built...")

Found potential sequences...
Genome DataFrame Built...


### Is this a STOP or not?
How many potential stops are there?
Can I visualize where I see them in the genome
Can I visualize where the actual stops are in the genome
Can I predict where the actual stops are in the genome and which frame the are?

In [113]:
## Find all potential stop locations
## Find closest in frame start
## Tag if its (in) a gene or not and which frame we have found it within

In [ ]:
# def pos_fix(reverse_end_indices,genome_length):
#     pos_list = [genome_length - indice - 1 for indice in reverse_end_indices]
#     return pos_list

# def stop_indices(genome):
#     stops = ["TAA","TAG","TGA"]
#     forward = [finds(genome,stop) for stop in stops]
#     reverse = [pos_fix(finds(reverse_complement(genome), stop),len(genome)) for stop in stops]
#     stop_locations = forward + reverse
#     return stop_locations

# def finds(seq,pattern):
#     """input: A sequence string (the genome) and a pattern sequence are
#     taken and all the patterns in the sequence are found
#     output: a list of start indices 
#     *** on this reverse strand... these are from the end of the sequence!!"""
#     starts = [m.start() for m in re.finditer(pattern, seq)]
#     return starts

In [282]:
import itertools
from itertools import product

def find_all_occurrences(pattern, string):
    """finds all occurrences of pattern in string, 
    returning a list of positions for each occurrence"""
    occurence_indices = [m.start() for m in re.finditer(pattern, string)]
    return occurence_indices

def stop_indices(seq):
    patterns = ["TAA","TAG","TGA"]
    stop_locations = [find_all_occurrences(pattern,seq) for pattern in patterns]
    stop_locations = list(itertools.chain(*stop_locations))
    return stop_locations

def start_indices(seq):
    pattern = "ATG"
    start_locations = find_all_occurrences(pattern,seq)
    return start_locations

def start_stops(genome):
    """input: output:"""
    stop_locations = sorted(stop_indices(genome))
    start_locations = start_indices(genome)
    return [start_locations, stop_locations]

# def split_into_frames(starts,stops):
#     """input: starts and stops are a list of indices 
#     where starts: (ATGs) or stops: (taa,tag,tga) where found in the genome.
#     output: a list of tuples in the form of [...(start, stop, frame),...]"""
#     res = []
#     for i in range(3):
#         s = np.array([-1] + [start for start in starts if (start - i) % 3 == 0])
#         e = [end for end in stops if (end - i) % 3 == 0]
#         indices = np.searchsorted(s,e)
#         res += [(s[ind-1],e[index]+3,i) for index,ind in enumerate(indices)]
#     return res

def split_into_frames(starts,stops):
    """input: starts and stops are a list of indices 
    where starts: (ATGs) or stops: (taa,tag,tga) where found in the genome.
    output: a list of tuples in the form of [...(start, stop, frame),...]"""
    res = []
    for i in range(3):
        s = np.array([-1] + [start for start in starts if (start - i) % 3 == 0])
        e = [end for end in stops if (end - i) % 3 == 0]
        indices = np.searchsorted(e,s)
        res += [(s[index],e[ind]+3,i) for index,ind in enumerate(indices)]
    return res

def process_genome(genome):
    """input:  
    output: """
    starts, stops = start_stops(genome)
    pairs = [pair for pair in split_into_frames(starts,stops) if pair[0] !=-1]
    pairs = [pair for pair in pairs if pair[1] - pair[0] > 200]
    return pairs


# def split_into_frames(starts,stops):
#     res = []
#     for i in range(3):
#         s = np.array([-1] + [start for start in starts if (start - i) % 3 == 0])
#         e = [end for end in stops if (end - i) % 3 == 0]
#         res += clever_name(s,e)
#     return res
    
    
# def clever_name(s,e):
#     indices = np.searchsorted(s,e)
#     return np.stack([s[indices-1], e]).T



In [299]:
## Pipeline
r_data = reverse_complement(data)

F_pairs = process_genome(data)
R_pairs = genome_end_genes(r_data)


In [300]:
import warnings
warnings.filterwarnings('ignore')

##Downsample from potential regions
potential_forward_stops = pd.DataFrame(F_pairs,columns=['start', 'end', 'frame'])
potential_subset = potential_forward_stops.sample(n=809)
potential_subset["seq"] = potential_subset.apply(lambda x: data[x[0]:x[1]],axis=1)
potential_subset["state"] = ['C']*len(potential_subset)
potential_subset

##Sample from Actual Sequences and label as true
actual_forward_stops = regions[regions.state == 'C']
actual_forward_stops["frame"] = actual_forward_stops.start.apply(lambda start: int(start) % 3)
actual_forward_stops["gene"] = True
actual_forward_stops.head()

##Merge actual and potential to create dataframe of both true and false sequences
merged_frames = pd.merge(potential_subset,actual_forward_stops, on=["start",'end','frame','state','seq'],how='outer')
merged_frames.gene.fillna(False, inplace=True)
merged_frames.gene.value_counts()

## regression - gene candidate and take nearest candiate neighbors or gene neighbors
## common sub-sequences in front of genes, 
## matrix of character combinations.... 
### Feature 1: nucleotide frequences
## Nucleotide Frequencies for each Potential Gene
res = [[l[1] for l in list(nucleotide_frequency(seg).items())] for seg in list(merged_frames.seq)]
frequency_df = pd.DataFrame(res)
frequency_df.columns=["a","c","g","t"]
print("Calculate Nucleotide Frequencies...")
print(frequency_df.head())

Calculate Nucleotide Frequencies...
          a         c         g         t
0  0.158120  0.259259  0.363248  0.219373
1  0.194313  0.254344  0.360190  0.191153
2  0.165192  0.333333  0.289086  0.212389
3  0.171198  0.303122  0.326284  0.199396
4  0.185417  0.320833  0.268750  0.225000


In [286]:
##Convert sequences in to amino acid sequences
def tri_split(seq):
    return ''.join([char+' ' if (i+1) % 3 == 0 else char for i,char in enumerate(seq)]).split()
def codonToAA(codon_list):
    aa_seq = ''.join([codon_to_aa[codon] for codon in codon_list])
    return aa_seq
def aa_frequency(seq):
    """Count the occurrences of amino acids in "seq"."""
    zeros = [0]*len(aa)
    counts = dict(zip(aa, zeros))
    for c in seq:
        counts[c] +=1
        
    total = sum(list(counts.values()))
    for key, value in counts.items():
        counts[key] = value / total
    return counts

def no_prior_stops(aa_seq):
    prior = aa_seq[0:len(aa_seq)-1]
    if 'x' in prior:
        return False
    else:
        return True
#     aa_frequency(codonToAA(tri_split(seq)))
    
aa_matrix = [list(aa_frequency(codonToAA(tri_split(seg))).values()) for seg in list(merged_frames.seq)]
# aa_matrix = [codonToAA(tri_split(seg)) for seg in list(merged_frames.seq)]
aa_frequency_df = pd.DataFrame(aa_matrix)
aa_frequency_df.columns=[['X','N','T','D','Y','L','W','Q','S','I','M','C','A','V','K','G','H','R','P','F','E']]
print("Calculated amino acid frequencies...")
print(aa_frequency_df.head())
# seq = 'ATGCTTCATTAACGTGTACAAAGACTAGGTTCCCTTCGCCTGCTTCACCGTCTTCCTCTTTAGTACTGTCGACCACCGTAAAGACATCATAGGCTTGATACTCACCTTTTTTGGCTGGCTCGATTAGCTCAAGCATGCCTTTATAAATGTCAGGATCAATCTTGCCGCCGCTCGTTAACATGTGGATGGTTTGCAAGTTAATCATTCGCTGTGTACGCTCTGCGGACACCTTAGCTAGTCCAGCAGCTGTTTGAGCAGTTTTGGCAGTCTTAGCGGTTTCCTGTGAGATTTTTTCAAGGTCGTCTACCTTTTGCACGGCTTCGCCCATTGCAATTTCAACGTATTCAGATTTTTTAAATTCTTCCAAAGTAGCTTTGATAATCTCTGTGTCATTATTTGAGGTTAAGTCCTGCTTGATTGGTTGAGAGATGACTGAGCCATCCTCAGCAGTGATAATAATATGTGTGCTTGCGACTGCTCCTGTGCTGTCAAATTGTGGGTATTTTCCTGTCACTTTCCAATTTCTCATGGTTATTCTCCTTTTTTACTTTCTTCAAATTGTTCCAAAATGTTGTCGATAAGAATGATTTCAGATGATGTAAATTCATCTTCACACTCTTCTAA'
# codonToAA(tri_split(seq))

### CountVectorizer N-gram analysis
### Features 2: amino acid frequencies
### CountVectorizer N-gram analysis



### Feature 3: length of sequence // false sequences should be at least 300 nucleotides long
## Add length of each sequence
merged_frames["len"] = merged_frames.apply(lambda x: x[1] - x[0],axis=1) #x[1] - x[0])
print("Calculated sequence lengths...")

## Combine tables
# 1. reset df index
merged_frames.reset_index(inplace=True)
# 2. concat
merged_frames = pd.concat([merged_frames,frequency_df,aa_frequency_df],axis=1)
print("Combined frequence and gene tables...")
len(merged_frames)

### linear regression on result and see if we can predict which STOPS are real given the prior sequences


### Views!
# draw where all actual stops are (color by frame)
# draw where all potential stops are (color by frame)

Calculated amino acid frequencies...
          X         N         T         D         Y         L         W  \
0  0.003215  0.045016  0.067524  0.045016  0.016077  0.112540  0.000000   
1  0.004016  0.040161  0.048193  0.024096  0.056225  0.152610  0.008032   
2  0.006329  0.075949  0.025316  0.075949  0.037975  0.069620  0.000000   
3  0.007246  0.050725  0.057971  0.007246  0.021739  0.152174  0.021739   
4  0.005051  0.035354  0.075758  0.050505  0.030303  0.126263  0.000000   

          Q         S         I    ...            C         A         V  \
0  0.054662  0.051447  0.064309    ...     0.006431  0.115756  0.070740   
1  0.080321  0.072289  0.076305    ...     0.004016  0.048193  0.040161   
2  0.025316  0.101266  0.132911    ...     0.025316  0.037975  0.025316   
3  0.036232  0.028986  0.036232    ...     0.000000  0.094203  0.094203   
4  0.045455  0.035354  0.045455    ...     0.020202  0.075758  0.065657   

          K         G         H         R         P         F

1528

In [287]:
merged_frames

,index,start,end,frame,seq,state,gene,len,a,c,...,C,A,V,K,G,H,R,P,F,E
0,0,1357297,1358230,1,ATGGTTTGCGGTTCTGGACTTAAAGCAATCCAATTAGCTGCGCAAA...,C,False,933,0.199357,0.315113,...,0.006431,0.115756,0.070740,0.054662,0.093248,0.019293,0.032154,0.038585,0.019293,0.064309
1,1,315250,315997,1,ATGAAGAGTTATATTGAACCTTTTATTGCTAGCAAGGCTTTATCTC...,C,True,747,0.160643,0.353414,...,0.004016,0.048193,0.040161,0.092369,0.044177,0.008032,0.040161,0.028112,0.056225,0.056225
2,2,443307,443781,0,ATGGGATATGACCAGAGAATAGGAAATTCGCACAATAATCCTTCTT...,C,False,474,0.124473,0.369198,...,0.025316,0.037975,0.025316,0.101266,0.050633,0.018987,0.044304,0.037975,0.056962,0.037975
3,3,214925,215339,2,ATGCCAGTTATGATTGCCTTATTCCAAGCCTTGACACGGGTATCTT...,C,False,414,0.188406,0.280193,...,0.000000,0.094203,0.094203,0.065217,0.021739,0.014493,0.072464,0.028986,0.072464,0.050725
4,4,1717369,1717963,1,ATGAAAACACACGGCAATGATCCAACAGAGTTGGCGCTGCTCAACC...,C,False,594,0.227273,0.314815,...,0.020202,0.075758,0.065657,0.075758,0.050505,0.080808,0.075758,0.040404,0.025253,0.030303
5,5,322934,323879,2,ATGAACCTCAAAGACAAGTTGGCGATTGTTTCAGCCCCTATTATCA...,C,False,945,0.172487,0.318519,...,0.000000,0.041270,0.060317,0.050794,0.050794,0.009524,0.038095,0.028571,0.041270,0.098413
6,6,322634,323879,2,ATGGCCTTAGTTTCTCTCAATCGTTCTCGGGTGGAACAAAAAGCAG...,C,False,1245,0.174297,0.302811,...,0.000000,0.055422,0.065060,0.050602,0.057831,0.009639,0.043373,0.026506,0.038554,0.084337
7,7,1258642,1259047,1,ATGAAGACGACATCTGGGTTATTTTCACCAGCACCTACCCAACGTT...,C,False,405,0.246914,0.281481,...,0.014815,0.088889,0.059259,0.059259,0.029630,0.007407,0.044444,0.133333,0.037037,0.014815
8,8,469411,470617,1,ATGGTTCTTGGGTTTGACCCTGCCTTTCGAACAGGTGCAAAATTGG...,C,False,1206,0.213101,0.283582,...,0.000000,0.092040,0.101990,0.064677,0.062189,0.022388,0.039801,0.044776,0.032338,0.057214
9,9,1258720,1259047,1,ATGGCAAAAGGAATGCCCAATTCTTTTTCAACAACGGAACCGACTT...,C,False,327,0.235474,0.281346,...,0.018349,0.073394,0.073394,0.064220,0.027523,0.009174,0.036697,0.137615,0.036697,0.018349


In [160]:
import pickle

class Pickler:
    def __init__(self, path):
        self.path = path

    def load(self):
        with open(self.path, 'rb') as f:
            return pickle.load(f)
    
    def save(self, thing):
        with open(self.path, 'wb') as f:
            pickle.dump(thing, f)

class StoredValue:
    def __init__(self, path, creator):
        self.pickler = Pickler(path)
        self.creator = creator
    def get(self):
        try:
            value = self.pickler.load()
        except FileNotFoundError:
            value = self.creator()
            pickler.save(value)
        return value

In [ ]:
def tri_split(seq):
    return ''.join([char+' ' if (i+1) % 3 == 0 else char for i,char in enumerate(seq)]).split()
def codonToAA(codon_list):
    aa_seq = ''.join([codon_to_aa[codon] for codon in codon_list])
    return aa_seq
def aa_frequency(seq):
    """Count the occurrences of amino acids in "seq"."""
    zeros = [0]*len(aa)
    counts = dict(zip(aa, zeros))
    for c in seq:
        counts[c] +=1
    return counts
seq = 'ATGCTTCATTAACGTGTACAAAGACTAGGTTCCCTTCGCCTGCTTCACCGTCTTCCTCTTTAGTACTGTCGACCACCGTAAAGACATCATAGGCTTGATACTCACCTTTTTTGGCTGGCTCGATTAGCTCAAGCATGCCTTTATAAATGTCAGGATCAATCTTGCCGCCGCTCGTTAACATGTGGATGGTTTGCAAGTTAATCATTCGCTGTGTACGCTCTGCGGACACCTTAGCTAGTCCAGCAGCTGTTTGAGCAGTTTTGGCAGTCTTAGCGGTTTCCTGTGAGATTTTTTCAAGGTCGTCTACCTTTTGCACGGCTTCGCCCATTGCAATTTCAACGTATTCAGATTTTTTAAATTCTTCCAAAGTAGCTTTGATAATCTCTGTGTCATTATTTGAGGTTAAGTCCTGCTTGATTGGTTGAGAGATGACTGAGCCATCCTCAGCAGTGATAATAATATGTGTGCTTGCGACTGCTCCTGTGCTGTCAAATTGTGGGTATTTTCCTGTCACTTTCCAATTTCTCATGGTTATTCTCCTTTTTTACTTTCTTCAAATTGTTCCAAAATGTTGTCGATAAGAATGATTTCAGATGATGTAAATTCATCTTCACACTCTTCTAA'

In [197]:
aa_frequency(codonToAA(tri_split(seq))).values()

dict_values([17, 3, 5, 4, 5, 35, 4, 16, 13, 6, 1, 10, 8, 9, 5, 4, 7, 21, 10, 23, 2])

In [198]:
aa_frequency(codonToAA(tri_split(seq))).keys()

dict_keys(['X', 'N', 'T', 'D', 'Y', 'L', 'W', 'Q', 'S', 'I', 'M', 'C', 'A', 'V', 'K', 'G', 'H', 'R', 'P', 'F', 'E'])

In [199]:
aa_frequency(codonToAA(tri_split(seq)))
## anomaly vs change point detection

{'A': 8,
 'C': 10,
 'D': 4,
 'E': 2,
 'F': 23,
 'G': 4,
 'H': 7,
 'I': 6,
 'K': 5,
 'L': 35,
 'M': 1,
 'N': 3,
 'P': 10,
 'Q': 16,
 'R': 21,
 'S': 13,
 'T': 5,
 'V': 9,
 'W': 4,
 'X': 17,
 'Y': 5}

In [143]:
mer.true = mer.true.fillna(False)
pd.isnull(mer.iloc[2].seq)
mer.iloc[2].seq is None
mer
# type(seq[1]),axis=1)#
mer.seq = mer.apply(lambda seq: seq[3] if not pd.isnull(seq[3]) else reverse_complement(data[int(seq[1]):int(seq[2])]), axis=1 )
mer["true"] = mer.true.fillna(False, inplace=True)
## regression - gene candidate and take nearest candiate neighbors or gene neighbors
## common sub-sequences in front of genes, 
## matrix of character combinations.... 
### Feature 1: nucleotide frequences
##Convert sequences in to amino acid sequences
### CountVectorizer N-gram analysis
### Features 2: amino acid frequencies
### CountVectorizer N-gram analysis

### linear regression on result and see if we can predict which STOPS are real given the prior sequences

AttributeError: 'DataFrame' object has no attribute 'true'

In [ ]:
### Views!
# draw where all actual stops are (color by frame)
# draw where all potential stops are (color by frame)

In [112]:
mer = pd.merge(potential_subset,actual_forward_stops, on=['start','end'], how="outer")


mer.true = mer.true.fillna(False)
pd.isnull(mer.iloc[2].seq)
mer.iloc[2].seq is None
mer
# type(seq[1]),axis=1)#
mer.seq = mer.apply(lambda seq: seq[3] if not pd.isnull(seq[3]) else reverse_complement(data[int(seq[1]):int(seq[2])]), axis=1 )
mer["true"] = mer.true.fillna(False, inplace=True)
## regression - gene candidate and take nearest candiate neighbors or gene neighbors
## common sub-sequences in front of genes, 
## matrix of character combinations.... 

In [97]:
mer.true = mer.true.fillna(False)
pd.isnull(mer.iloc[2].seq)
mer.iloc[2].seq is None
mer
# type(seq[1]),axis=1)#
mer.seq = mer.apply(lambda seq: seq[3] if not pd.isnull(seq[3]) else reverse_complement(data[int(seq[1]):int(seq[2])]), axis=1 )
mer["true"] = mer.true.fillna(False, inplace=True)

In [44]:
mer["true"] = mer.true.fillna(False, inplace=True)

In [77]:
regions["true"] = True
merged_df = pd.merge(df,regions[regions.state != "N"], on=['start', 'end'],how='outer')

In [78]:
merged_df["true"] = merged_df.true.fillna(False)

In [175]:
# merged_df[merged_df.true == True]

In [22]:
## Add length of each sequence
df["len"] = df.apply(lambda x: x[1] - x[0],axis=1) #x[1] - x[0])
print("Calculated sequence lengths...")

Calculated sequence lengths...


In [26]:
df[df.strand == 'F'][["start","end"]].head()

,start,end
0,37,112
1,76,112
2,188,347
3,231,1587
4,241,292


In [116]:
## Obtain potential gene sequence  -- TIME SUCK
df["seq"] = df.apply(lambda x: data[x[0]:x[1]],axis=1)
print("Obtained potential sequences...")

## Nucleotide Frequencies for each Potential Gene
res = [[l[1] for l in list(nucleotide_frequency(seg).items())] for seg in list(df.seq)]
frequency_df = pd.DataFrame(res)
frequency_df.columns=["a","c","g","t"]
print("Calculate Nucleotide Frequencies...")

## Combine tables
# 1. reset df index
df.reset_index(inplace=True)
# 2. concat
df = pd.concat([df,frequency_df],axis=1)
print("Combined frequence and gene tables...")

KeyboardInterrupt: 

In [66]:
## Filter for Genes
    # x[0] - start
    # x[1] - end
    # x[4] - strand
# df["is_gene"] = df.apply(lambda x: check_gene_state([x[0],x[1]],annot, x[4]), axis=1)

In [341]:
y = nsample_df.is_gene
x = nsample_df[["start","end","shift","len","a","c","g","t"]]

In [357]:
nsample_df.is_gene.value_counts()

False    9995
True        5
Name: is_gene, dtype: int64

In [326]:
set(nsample_df.isnull().any(axis=1))

{False}

In [290]:
sample_df["len"] = sample_df.seq.apply(lambda x: len(x))


In [291]:
Nsample_df = sample_df.reindex(np.arange(10000))

In [296]:
set(sample_df.isnull().any(axis=1))

{False}

In [283]:
ex = Nsample_df.seq
# ex.apply(lambda x: len(str(x)))
sample_df.isnull().shape

(10000, 7)

In [361]:
xxx = list(sample_df[sample_df.is_gene == True].seq)

In [362]:
import swalign
# choose your own values here… 2 and -1 are common.
match = 2
mismatch = -1
scoring = swalign.NucleotideScoringMatrix(match, mismatch)

sw = swalign.LocalAlignment(scoring)  # you can also choose gap penalties, etc...
alignment = sw.align(xxx[0],xxx[1])
alignment.dump()

Query:    1 ATG--ATTC--AAT--T----TTCAA-TTAAT--CGCACATTATTTAT--T--C-ATGCTTT--AA-ATACAACTAAACGTGCTATT-AGCACTAAA-AATG-CC--ATTCCTATTCTTTCATCAA--TAA--A--AA---TTGAAGTCACTT--CTACAGGAGTAACTTTAACAGGGTCT-AACGGTC-AAATATCA--ATTGAAAACACTATTCCTGTAAGTA-ATGAAAAT-GCT-GG-T-TTGCTAATTAC-CTCTCCAGGAGCTATTTTA-T--TAGAAGCTA-GT-T-TTTTTATTAATATTATTTCAAGTTTG-CCAGATATTAGTATAA---ATGT--TAAA-GAAATT-G-AA--CAACACCAAGTT-GTTTTAA-CC-AGTGGT-AAATCAGAGAT-T-AC--CTTAAAA--GGA--AAAGATG-TTG--A-CCAGTA-TC-CTCGT-CTA-CA---A-GAAGTATCAACAG-AAAATCCTTTG-ATTTTA-A-AAACAAAATTATTGAAG-TCT---AT-TATT-GCT-GAAACA-GCT-TTTGCAGCCAGTTTACAAGAA-AG-TC-GTCCTA-TTTTAACAGGAGTTCATATTGTATTAAGTAATCATAA-AGATT-T--TAA--AGC-A--GTAGCGA-CTGA-C-TCTC-ATCG-TAT-G-AGCCAACGTTTAATCACT--TTGGACAAT----ACTT--CAG-CAGAT-T------T--TGATGTAGTT-ATTC-CAAGT--AAATCTTTGAGAGAATTTTCAGCAGTATTTACAGATGATATTGAGACCGTTGAGGTATTTTT-CTCACCA-AGCCA---A--ATCTTGTTCAGAAGTGAACACATTT-CTTTTTAT-ACACG-CCTCTTAGAAGG-AAATTATCCCGATA-CAGAC-CGTT--T-A-TT-AATGACAGAATTTGAGACGGAGGT--TGTTTTCAATAC--CC-AATCCCTTCGC-CAC-G

In [301]:
frequency_df["start"] = sample_df.start

In [304]:
frequency_df.drop(frequency_df.start)

ValueError: labels [ nan  nan  nan ...,  nan  nan  nan] not contained in axis

In [258]:
frequency_df.shape

(10000, 4)

In [259]:
new_df = pd.concat([Nsample_df,frequency_df],axis=1)

In [260]:
new_df.shape

(10000, 11)

In [ ]:
nucleotide_frequency(seq)

In [201]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(1,3), min_df= 1)
d = vectorizer.fit_transform(list(sample_df['seq']))


In [98]:
check_gene_state((37, 1438),annot,'C','F')

False

In [87]:
sample_df.iloc[1].seq

u'ATGAAAAATAGTTGAAAACAATAGCGGTGTCCCCTTAAAATGGCTTTTCCACAGGTTGTGGAGAACCCAAATTAACAGTGTTAATTTATTTTCCACAGGTTGTGGAAAAACTAACTATTATCCATCGTTCTGTGGAAAACTAGAATAGTTTATGGTAGAATAGTTCTAGAATTATCCACAAGAAGGAACCTAGTATGA'

In [ ]:
df["sequence_s"] =  df[['start', 'end']].apply(lambda x: data[x[0]:x[1]], axis=1)

In [39]:
result = [[l[1] for l in list(nucleotide_frequency(seg).items())] for seg in seg_50]
df = pd.DataFrame(result)
df.columns=["a","c","g","t"]
df["gene"] = gene_label(seg_50_an)

gene_df = pd.DataFrame({"label":df.gene,"sequence": seg_50})

50


In [124]:
str(u'x')

'x'

In [122]:
he = prior_state_emissions(data,annot)

In [123]:
he

{u'C': ['C',
  0,
  {'A': 0.3129376364986609,
   'C': 0.18263194813359745,
   'G': 0.20960154402206216,
   'T': 0.2948288713456795},
  {'C': 0.39282114788001343, 'N': 0.0004367210615614748, 'R': 0.0}],
 u'N': ['N',
  1,
  {'A': 0.3190287760527498,
   'C': 0.1789016522921669,
   'G': 0.18693843939846827,
   'T': 0.315131132256615},
  {'C': 0.0004367210615614748,
   'N': 0.2719266092685273,
   'R': 0.000344950257525071}],
 u'R': ['R',
  0,
  {'A': 0.29606142439037003,
   'C': 0.20997386780698055,
   'G': 0.1825552662752494,
   'T': 0.3114094415274},
  {'C': 0.0, 'N': 0.000344950257525071, 'R': 0.3336883603850271}]}

In [ ]:
emi = emissions(data,annot)

In [100]:
trs = transitions(annot)

In [ ]:
# dir(hmm)
from hmm import hmm

In [133]:
he[u'N'][2]

{'A': 0.3190287760527498,
 'C': 0.1789016522921669,
 'G': 0.18693843939846827,
 'T': 0.315131132256615}

In [136]:
NC = hmm.state(he[u'N'][0],he[u'N'][1],he[u'N'][2],he[u'N'][3])
C = hmm.state(he[u'C'][0],he[u'C'][1],he[u'C'][2],he[u'C'][3])
R = hmm.state(he[u'R'][0],he[u'R'][1],he[u'R'][2],he[u'R'][3])
gene_model = hmm.hmm(['A','T','G','C'],[NC,C,R])

In [139]:
gene_model.enumerate('ATGC')

('N', 'C', 'C', 'C'): -6.615035


ValueError: math domain error

In [83]:
s1 = hmm.state(
        'S1',            # name of the state
        0.5,             # probability of being the initial state
        { '1': 0.5,      # probability of emitting a '1' at each visit
          '2': 0.5 },    # probability of emitting a '2' at each visit
        { 'S1': 0.9,     # probability of transitioning to itself
          'S2': 0.1 })   # probability of transitioning to state 'S2'
s2 = hmm.state('S2', 0.5,
        { '1': 0.25, '2': 0.75 },
        { 'S1': 0.8, 'S2': 0.2 })
model = hmm.hmm(['1', '2'],  # all symbols that can be emitted
                [s1, s2])    # all of the states in this HMM

In [ ]:
model = hmm.hmm(['A', 'C','T','G'],  # all symbols that can be emitted
                [s1, s2])

In [84]:
model.enumerate('222')

('S2', 'S2', 'S2'): -2.073786
('S2', 'S2', 'S1'): -1.647817
('S2', 'S1', 'S2'): -1.948847
('S2', 'S1', 'S1'): -1.170696
('S1', 'S2', 'S2'): -2.550907
('S1', 'S2', 'S1'): -2.124939
('S1', 'S1', 'S2'): -2.073786
('S1', 'S1', 'S1'): -1.295635
BEST: ('S2', 'S1', 'S1'): -1.170696


In [45]:
training_data = build_training_data(annot,data)

In [58]:
import hmm

In [57]:
tabul = hmm(data, annot)

TypeError: 'module' object is not callable

In [54]:
train_hmm(training_data)

NameError: global name 'state' is not defined

In [ ]:
emissions(data,annot)

In [ ]:
set(data)

In [ ]:
# two_state_transitions(annot,1)
two_state_emissions(data,annot,1)
# str(annot[0]) == "N"

In [ ]:
reverse_complement("aaatttgggcca")

In [ ]:
df["sequence"] = gene_df.sequence
df["start"] = gene_df.sequence.apply(lambda seq: "ATG" in seq)
df["stop"] = gene_df.sequence.apply(lambda seq: any(x in seq for x in ["TAA", "TAG", "TGA"]))
# df.iloc[0].sequence

In [ ]:
# from collections import defaultdict
def aa_find(seq):
    codon_count = {}
    for aa, codons in codon_map.items():
        codon_count[aa] = 0
        if any(x in seq for x in codons):
            codon_count[aa] += 1
    return codon_count

In [ ]:
df["dict"] = gene_df.sequence.apply(lambda seq: aa_find(seq)) 

In [ ]:
aa_counts = pd.DataFrame(list(df["dict"]))

In [ ]:
# aa_counts[["a"],["c"],["g"],["t"],["gene"]] = [[df.a],[df.c],[df.g],[df.t],[df.gene]]
# aa_counts[["a"]]
aa_counts["a"] = df.a
aa_counts["c"] = df.c
aa_counts["g"] = df.g
aa_counts["t"] = df.t
aa_counts["gene"] = df.gene


In [ ]:
df.shape

In [ ]:
a = []
a.append(aa_find(df.iloc[1].sequence).values())
a.append(aa_find(df.iloc[2].sequence).values())
a.append(aa_find(df.iloc[3].sequence).values())
pd.DataFrame(a)

In [ ]:
[a[1] for a in aa_find(df.iloc[1].sequence).items()]

In [ ]:
aa_find(df.iloc[1].sequence).items()

In [ ]:
codon_to_aa.items()
codon_map.items()


## Feature Development

In [ ]:
# ngrams
- 3 grams, which frame?, relationship between subsequent sequence fragments.
... am I next to a fragment

In [ ]:
"atg" in "gowijgoijatgsoijdfo"

In [ ]:
gene_df.sequence[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(1,3), min_df= 1)
d = vectorizer.fit_transform(list(gene_df['sequence']))

In [ ]:
def frameShift(seq,shift):
    new_seq = ''
    for j, i in enumerate(seq):
        if j % shift == 0:
            new_seq = new_seq + i + ' '
        else: 
            new_seq = new_seq + i
    return new_seq


fs = [frameShift(seq,3) for seq in gene_df.sequence]

In [ ]:
fs[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char_wb', ngram_range=(3,3), min_df= 1)
d = vectorizer.fit_transform(fs)

In [ ]:
fs[0]

In [ ]:
x = vectorizer.vocabulary_.copy()
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())

In [ ]:
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())

In [ ]:
my_dict2

In [ ]:
nx = [x.pop(val) for val in vectorizer.vocabulary_ if len(val.strip()) != 3]


In [ ]:
gg = list(d)

In [ ]:
len(gg)

In [ ]:
len(d.indptr)
len(d.data)

In [ ]:
test = gg[0]

In [ ]:
import pandas as pd

l = []
for i in range(len(gg)):
    doc = gg[i]
    l.append(pd.DataFrame({"doc":i,"index":doc.indices,"count":doc.data}))
dum = pd.concat(l)
# df2 = pd.DataFrame({"index":doc.indices,"count":doc.data})
# df2


In [ ]:
len(set(dum.doc))

In [ ]:
dum.index

In [ ]:
x = vectorizer.vocabulary_.copy()
my_dict2 = dict((y,str(x)) for x,y in x.iteritems())
dum["codon"] = dum['index'].apply(lambda ind: my_dict2[ind])
codonCount = dum[dum.codon.apply(lambda v: len(v.strip())) == 3]

In [ ]:
codonCount = dum[dum.codon.apply(lambda v: len(v.strip())) == 3]
# dum.codon.apply(lambda v: len(v.strip()))

In [ ]:
codonCount

In [ ]:
ngrams = list(vectorizer.get_feature_names())
[gram.strip() for gram in ngrams if len(gram.strip()) == 3]




In [ ]:
df

In [ ]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
codon_to_aa = invert(codon_map)

def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq

In [ ]:
codon_to_aa

#e

In [ ]:
def codonToAA(strand):
    aa_seq = ''.join([codon_to_aa(codon) for codon in strand])
    return aa_seq


In [ ]:
new_seq

In [ ]:
ngrams = list(vectorizer.get_feature_names())
ngrams


In [ ]:
## amino acid to nucleotide chart http://www.cbs.dtu.dk/courses/27619/codon.html
aa = ["I","L","V","F","M","C","A","G","P","T","S","Y","W","Q","N","H","E","D","K","R","STOP"]
aa_name = ["Isoleucine","Leucine","Valine","Phenylalanine","Methionine","Cysteine","Alanine",
           "Glycine","Proline","Threonine","Serine","Tyrosine","Tryptophan",
           "Glutamine","Asparagine","Histidine","Glutamic_acid", "Aspartic_acid", 
           "Lysine","Arginine","Stop_codons"]
codons = [["ATT", "ATC", "ATA"],["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          ["GTT", "GTC", "GTA", "GTG"],["TTT", "TTC"],["ATG"],["TGT", "TGC"],
          ["GCT", "GCC", "GCA", "GCG"],["GGT", "GGC", "GGA", "GGG"],
          ["CCT", "CCC", "CCA", "CCG"],
          ["ACT", "ACC", "ACA", "ACG"],["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          ["TAT", "TAC"],["TGG"],["CAA", "CAG"],["AAT", "AAC"],["CAT", "CAC"],["GAA", "GAG"],
          ["GAT", "GAC"],["AAA", "AAG"],["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],
          ["TAA", "TAG", "TGA"]]

codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q"["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
# aa_properties
Aliphatic: Alanine,Isoleucine,Leucine,Valine
Aromatic: Phenylalanine, Tryptophan, Tyrosine
Polar_neutral: Asparagine, Cysteine, Glutamine, Methionine,Serine,Threonine
charged: Aspartic acid, Glutamic acid
    
Properties table
https://www.thermofisher.com/us/en/home/life-science/protein-biology/protein-biology-learning-center/protein-biology-resource-library/pierce-protein-methods/amino-acid-physical-properties.html

http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3456822/pdf/10867_2004_Article_396406.pdf

http://www.proteinstructures.com/Structure/Structure/amino-acids.html

In [ ]:
codon_map = {"I":["ATT", "ATC", "ATA"],"L":["CTT", "CTC", "CTA", "CTG", "TTA"," TTG"],
          "V":["GTT", "GTC", "GTA", "GTG"],"F":["TTT", "TTC"],"M":["ATG"],"C":["TGT", "TGC"],
          "A":["GCT", "GCC", "GCA", "GCG"],"G":["GGT", "GGC", "GGA", "GGG"],
          "P":["CCT", "CCC", "CCA", "CCG"],
          "T":["ACT", "ACC", "ACA", "ACG"],"S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"],
          "Y":["TAT", "TAC"],"W":["TGG"],"Q":["CAA", "CAG"],"N":["AAT", "AAC"],"H":["CAT", "CAC"],
             "E":["GAA", "GAG"],"D":["GAT", "GAC"],"K":["AAA", "AAG"],
             "R":["CGT", "CGC", "CGA", "CGG","AGA", "AGG"],"Stop":["TAA", "TAG", "TGA"]}
def invert(d):
    return dict( (v,k) for k in d for v in d[k] )
codon_to_aa = invert(codon_map)

In [ ]:
codon_to_aa

## Model testing

In [288]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, classification_report


from sklearn.learning_curve import learning_curve

from sklearn.cross_validation import train_test_split, KFold, cross_val_score

In [292]:
# x = df[["a","c","g","t","start","stop"]]
y = merged_frames.gene
x = merged_frames[['frame', 'len', 'a',
       'c', 'g', 't', 'X', 'N', 'T', 'D', 'Y', 'L', 'W', 'Q', 'S', 'I', 'M',
       'C', 'A', 'V', 'K', 'G', 'H', 'R', 'P', 'F', 'E']]
# y = df.gene
# x = aa_counts.drop('gene', 1)

In [294]:
y = y.apply(lambda x: int(x))

In [298]:
algorithms = [RandomForestClassifier(), DecisionTreeClassifier(), GaussianNB(), 
              SVC(), KNeighborsClassifier(n_neighbors=6), LogisticRegression()]

for algo in algorithms:
    accuracy = cross_val_score(algo,x,y)
    print("{:s} Accuracy Score : {:f}".format(str(algo).split('(', 1)[0],accuracy.mean()))

RandomForestClassifier Accuracy Score : 0.540549
DecisionTreeClassifier Accuracy Score : 0.530125
GaussianNB Accuracy Score : 0.583099
SVC Accuracy Score : 0.550363
KNeighborsClassifier Accuracy Score : 0.524844
LogisticRegression Accuracy Score : 0.594902


In [52]:
import hmm